<a href="https://colab.research.google.com/github/ElenaPribylova/-10/blob/main/2_%D0%BA%D0%B5%D0%B9%D1%81_%D0%9F%D0%95%D0%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2025-10-08 22:47:07--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-10-08 22:47:07--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2025-10-08 22:47:08 (33.6 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Небольшое описание столбцов в таблицах:  
**Лист1:** `student_id` - айди студента; `student_name` - имя студента; `installment` (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).  
**Лист2:** `student_id` - айди студента; `last_payment_date` - дата последней полученной оплаты; `expected_payment_date` - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).  
**Лист3:** `student_id` - айди студента; `already_payed_amount` - уже выплаченная часть рассрочки; `left_to_pay` - оставшаяся (невыплаченная) часть; `one-time_payment` - единоразовый платеж; `installment_amount` - общая сумма, которая бралась в рассрочку.

Как примерно должен выглядеть результат:

In [ ]:
with open('result.txt') as f:
  print(f.read())

In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023). При расчетах отталкивайтесь от даты ожидаемого платежа, разницу между платежами берите 183 дня.

**Решение**

Напишите свое решение ниже

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('/content/creds.json', scope)

client = gspread.authorize(creds)

sheet = client.open("Installments").worksheet("Лист1")
sheet1_data = sheet.get_all_records()

sheet = client.open("Installments").worksheet("Лист2")
sheet2_data = sheet.get_all_records()

sheet = client.open("Installments").worksheet("Лист3")
sheet3_data = sheet.get_all_records()

WorksheetNotFound: Лист1

In [ ]:
from datetime import datetime
from datetime import timedelta
import math

def generate_report(sheet1, sheet2, sheet3):
    report = {}

    # Шаг 1: Соберем данные с трех листов и сохраним в словарь
    for student in sheet1:
        student_id = student['student_id']
        name = student['student_name']
        installment = student['installment']

        report[student_id] = {'name': name, 'debt': 0, 'last_payment_date': None}

    for payment in sheet2:
        student_id = payment['student_id']
        last_payment_date = datetime.strptime(payment['last_payment_date'], '%d.%m.%Y')
        expected_payment_date = datetime.strptime(payment['expected_payment_date'], '%d.%m.%Y')
        if student_id in report:
            report[student_id]['last_payment_date'] = last_payment_date
            report[student_id]['expected_payment_date'] = expected_payment_date

    for debt in sheet3:
        student_id = debt['student_id']
        already_payed_amount = debt['already_payed_amount']
        left_to_pay = debt['left_to_pay']
        one_time_payment = debt['one-time_payment']
        installment_amount = debt['installment_amount']

        # Проверяем просрочен ли ожидаемый платеж. Учитываем несколько просрочек.
        if student_id in report and report[student_id]['expected_payment_date'] < datetime(2023, 3, 1):
            period = datetime(2023, 3, 1) - report[student_id]['expected_payment_date']
            debt = one_time_payment * (math.ceil(period/timedelta(days=183)))
            if debt <= left_to_pay:
                report[student_id]['debt'] = debt
            else:
                report[student_id]['debt'] = left_to_pay

    # Шаг 2: Запишем отчет в файл
    with open("student_debt_report.txt", "w") as file:
        for student_id, data in report.items():
            if data['debt'] == 0:
                continue
            else:
                name = data['name']
                debt = data['debt']
                file.write(f"Студент {name} - долг {debt} рублей\n")

generate_report(sheet1_data, sheet2_data, sheet3_data)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [9]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2025-10-11 10:02:51--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.116.3
Connecting to gist.github.com (gist.github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2025-10-11 10:02:51--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt.3’

student_debt.txt.3  100%[===================>]  11.06K  --.-KB/s    in 0s      

2025-10-11 10:02:52 (23.2 MB/

FileNotFoundError: [Errno 2] No such file or directory: 'student_debt_report.txt'

In [10]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

AssertionError: При проверке возникла ошибка NameError("name 'user_answer' is not defined")